# Model 4 - 3D CNN

In [1]:
# FOR GOOGLE COLAB
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd My Drive

/content/drive/My Drive


In [4]:
cd Dissertation/5_Code/1_code

/content/drive/My Drive/Dissertation/5_Code/1_code


In [5]:
ls

 DataPreparation.ipynb          Model5_ANN.ipynb
'EDA&PreProcessing.ipynb'       my_model_16_16_16.h5
 keras-test.ipynb               original-voxel-model.ipynb
 Model1_PointNetFull.ipynb      PointNetBasic.ipynb
 Model2_PointNetBasic.ipynb     Py/
 Model3_PointNetBasic_l.ipynb   __pycache__/
 Model4_3DCNN.ipynb             voxel-model.ipynb


## 1. Set Seed for Reproducibility

In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


## 2. Install dependencies

In [3]:
# Install depedencies
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report
#from mlxtend.plotting import plot_confusion_matrix
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta, SGD, Adam
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

## 3. Load data

In [4]:
size = 16
h,w,d = size,size,size

if size == 16:
    X = np.load('../2_pipeline/voxeldata_16.npy')
else:
    X = np.load('../2_pipeline/voxeldata_32.npy')
y = np.load('../2_pipeline/labels.npy')

#split the data
print("Splitting the data...")
from sklearn.model_selection import train_test_split
#split data into 1: train+validation set and 2: test set 
X_train_val, X_test, y_train_val, y_test = \
train_test_split(X, y, random_state=0, test_size=0.2)

# split train+validation set into 1a) training and 1b) validation sets
X_train, X_val, y_train, y_val = \
train_test_split(X_train_val, y_train_val, random_state=1, test_size=0.2)

# Check train and test size
print("X training size is: ", X_train.shape)
print("y training size is: ", y_train.shape)
print("\nX val size is: ", X_val.shape)
print("y val size is: ", y_val.shape)
print("\nX test size is: ", X_test.shape)
print("y test size is: ", y_test.shape)

x_train = X_train.reshape(X_train.shape[0], h, w, d,1)
x_val = X_val.reshape(X_val.shape[0], h, w, d,1)
x_test = X_test.reshape(X_test.shape[0], h, w, d,1)

Splitting the data...
X training size is:  (182, 4096)
y training size is:  (182,)

X val size is:  (46, 4096)
y val size is:  (46,)

X test size is:  (58, 4096)
y test size is:  (58,)


## 4. Build 3D CNN
Lets create the model architecture. The architecture is described below:

Input and Output layers:

* One Input layer with dimentions 16, 16, 16, 3
* Output layer with dimensions 2

Convolutions :
* Apply 4 Convolutional layer with increasing order of filter size (standard size : 8, 16, 32, 64) and fixed kernel size = (3, 3, 3)
* Apply 2 Max Pooling layers, one after 2nd convolutional layer and one after fourth convolutional layer.

MLP architecture:
* Batch normalization on convolutiona architecture
* Dense layers with 2 layers followed by dropout to avoid overfitting

In [45]:
# Hyper parameters
max_epochs = 25
batch_size = 16 # 8 < 16 > 32 > 128
#opt = Adadelta(lr=0.001)
opt = Adam(lr=0.001, decay=0.7)
dropout_rate=0.2

#from keras.optimizers import SGD
#opt = SGD(lr=0.001, momentum=0.9)

# Class weights
class_weight = {0: 0.3,
                1: 0.7}

model = Sequential()
# Convolution layers
model.add(Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu', input_shape=(h, w, d, 1)))
model.add(Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu'))
# Add max pooling to obtain the most informatic features
model.add(MaxPool3D(pool_size=(2, 2, 2)))

# Convolution layers
model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu'))
model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'))
# Add max pooling to obtain the most informatic features
model.add(MaxPool3D(pool_size=(2, 2, 2)))

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
model.add(BatchNormalization())
model.add(Flatten())

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
model.add(Dense(units=(h*w*d), activation='relu'))
model.add(Dropout(dropout_rate))
model.add(BatchNormalization())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_57 (Conv3D)           (None, 14, 14, 14, 8)     224       
_________________________________________________________________
conv3d_58 (Conv3D)           (None, 12, 12, 12, 16)    3472      
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 6, 6, 6, 16)       0         
_________________________________________________________________
conv3d_59 (Conv3D)           (None, 4, 4, 4, 32)       13856     
_________________________________________________________________
conv3d_60 (Conv3D)           (None, 2, 2, 2, 64)       55360     
_________________________________________________________________
max_pooling3d_30 (MaxPooling (None, 1, 1, 1, 64)       0         
_________________________________________________________________
batch_normalization_43 (Batc (None, 1, 1, 1, 64)       256       
__________

## 5. Train model & Evaluate Model

In [46]:
print("\n####################### Training Model ##############################")
print("Training...")
history = model.fit(x=x_train, y=y_train, batch_size=batch_size, \
                    epochs=max_epochs, class_weight=class_weight, \
                    validation_data=(x_val, y_val), verbose=1)


####################### Training Model ##############################
Training...
Train on 182 samples, validate on 46 samples
Epoch 1/25
182/182 [==============================] - 10s 54ms/step - loss: 0.4495 - acc: 0.5330 - val_loss: 1.4487 - val_acc: 0.4348
Epoch 2/25
182/182 [==============================] - 2s 13ms/step - loss: 0.2562 - acc: 0.6703 - val_loss: 1.1851 - val_acc: 0.4348
Epoch 3/25
 96/182 [==============>...............] - ETA: 1s - loss: 0.1809 - acc: 0.7500

KeyboardInterrupt: 

In [44]:
print("\n###################### Model Performance ############################")
# Make predictions 
y_pred = model.predict(x_test)

# evaluate the model
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('\nTrain: %.3f, Test: %.3f' % (train_acc, test_acc))
print("\n#####################################################################")

#Classification report
report = classification_report(y_test, y_pred.round())
print("\nClassfication Report for test:\n", report)
print("\n#####################################################################")


###################### Model Performance ############################

Train: 0.571, Test: 0.466

#####################################################################

Classfication Report for test:
               precision    recall  f1-score   support

         0.0       0.73      0.22      0.34        36
         1.0       0.40      0.86      0.55        22

   micro avg       0.47      0.47      0.47        58
   macro avg       0.57      0.54      0.45        58
weighted avg       0.60      0.47      0.42        58


#####################################################################


In [26]:
# Create the confusion matrix
cm = confusion_matrix(y_true = y_test, y_pred = y_pred.round())
print("\nOur test confusion matrix yields: ")
fig, ax = plot_confusion_matrix(conf_mat=cm)
plt.show()
print("\n#####################################################################")


# Plot AUC
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc = auc(fpr, tpr)

print("\nThe AUC is", auc)
# Create AUC plot
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()
print("\n#####################################################################")


Our test confusion matrix yields: 


NameError: name 'plot_confusion_matrix' is not defined

In [0]:
from keras.models import load_model
#model.save('my_model_16_16_16.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [0]:
# returns a compiled model
# identical to the previous one
#model = load_model('my_model_16_16_16.h5')